In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
mnist=tf.keras.datasets.mnist
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()

train_num=int(len(train_images)*0.8)
train_x=train_images[:train_num]
train_y=train_labels[:train_num]
vaild_x=train_images[train_num:]
vaild_y=train_labels[train_num:]
test_x=test_images
test_y=test_labels

train_x=train_x.reshape(-1,784)
train_y=tf.one_hot(train_y,depth=10)
vaild_x=vaild_x.reshape(-1,784)
vaild_y=tf.one_hot(vaild_y,depth=10)
test_x=test_x.reshape(-1,784)
test_y=tf.one_hot(test_y,depth=10)

train_x=tf.cast(train_x/255.0,tf.float32)
vaild_x=tf.cast(vaild_x/255.0,tf.float32)
test_x=tf.cast(test_x/255.0,tf.float32)

w=tf.Variable(tf.random.normal([784,10],mean=0.0,stddev=1.0,dtype=tf.float32))
b=tf.Variable(tf.zeros([10]),dtype=tf.float32)

2023-03-09 19:12:01.996010: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def plt_plot(image):
    plt.imshow(image.reshape(28,28),cmap='binary')
    plt.show()
def model(x,w,b):
    pred=tf.matmul(x,w)+b
    return tf.nn.softmax(pred)
def loss_function(x,y,w,b):
    pred=model(x,w,b)
    loss_=tf.keras.losses.categorical_crossentropy(y_true=y,y_pred=pred)
    return tf.reduce_mean(loss_)
def grad(x,y,w,b):
    with tf.GradientTape() as tape:
        return tape.gradient(loss_function(x,y,w,b),[w,b])
def accuracy(x,y,w,b):
    pred=model(x,w,b)
    correct_prediction=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    return tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [4]:
learning_rate=0.001
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)

loss_list_train=[]
loss_list_vaild=[]
acc_list_train=[]
acc_list_vaild=[]

In [5]:
training_epochs=5
batch_size=50
total_step=int(train_num/batch_size)
for epoch in range(training_epochs):
    for step in range(total_step):
        xs=train_x[step*batch_size:(step+1)*batch_size]
        ys=train_y[step*batch_size:(step+1)*batch_size]

        grads=grad(xs,ys,w,b)
        optimizer.apply_gradients(zip(grads,[w,b]))
    
    loss_list_train.append(loss_function(train_x,train_y,w,b).numpy())
    loss_list_vaild.append(loss_function(vaild_x,vaild_y,w,b).numpy())
    acc_list_train.append(accuracy(train_x,train_y,w,b).numpy())
    acc_list_vaild.append(accuracy(vaild_x,vaild_y,w,b).numpy())
    print('epoch={:2d}, train loss={:.4f}, train accuracy={:.4f}, value loss={:.4f}, value accuracy=:{:.4f}'
    .format(epoch+1,loss_list_train[-1],acc_list_train[-1],loss_list_vaild[-1],acc_list_vaild[-1]))

epoch= 1, train loss=1.8203, train accuracy=0.6781, value loss=1.6296, value accuracy=:0.7007
epoch= 2, train loss=1.0656, train accuracy=0.7910, value loss=0.9390, value accuracy=:0.8106
epoch= 3, train loss=0.8188, train accuracy=0.8323, value loss=0.7306, value accuracy=:0.8466
epoch= 4, train loss=0.6912, train accuracy=0.8537, value loss=0.6272, value accuracy=:0.8628
epoch= 5, train loss=0.6098, train accuracy=0.8675, value loss=0.5630, value accuracy=:0.8740
